<a href="https://colab.research.google.com/github/Adem-HB/Satisfaction_client/blob/main/scarping_avis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pdb
import re
import os
import random
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0"}
base_url = 'https://fr.trustpilot.com/review/troicet.com'

In [3]:
def fetch_url(url, headers, max_attempts=10):
    for attempt in range(max_attempts):
        timeout = random.randint(1, 10)
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()  # Raise en exception pour les HTTP erreurs
            print(f'Successfully fetched {url}')
            return response
        except requests.RequestException as e:
            print(f'Attempt {attempt + 1} failed for {url} with timeout {timeout}: {e}')
    return None

In [4]:
response = fetch_url(base_url, headers)
contenu = response.text

Successfully fetched https://fr.trustpilot.com/review/troicet.com


In [5]:
def extract_comments_and_total(contenu):
    comments = re.findall('"reviewBody":"(.*?)"', contenu)
    # Exemple : il faut peut-être ajuster le pattern selon la page réelle
    nb_views = re.search(r"(?:[\d\s]+) sur ([\d\s]+) avis", contenu)
    nb_views = int(nb_views.group(1).replace('\u202f', '').replace(' ', ''))

    return comments, nb_views


In [6]:
# Page 1
import math

comments, total_reviews = extract_comments_and_total(contenu)

all_comments = comments.copy()

if total_reviews:
    total_pages = math.ceil(total_reviews / 20)
else:
    total_pages = 1

print(f"Nombre total d'avis : {total_reviews} => {total_pages} pages estimées.")

# Pages 2 à total_pages
for page in range(2, total_pages + 1):
    url = f"{base_url}?page={page}"
    page_comments = re.findall(r'"reviewBody":"(.*?)"', response.text)
    if not page_comments:
        print(f"Aucun commentaire trouvé à la page {page}, arrêt.")
        break
    all_comments.extend(page_comments)
    print(f"Page {page} : {len(page_comments)} commentaires trouvés.")

print(f"\nTotal de commentaires récupérés : {len(all_comments)}")

Nombre total d'avis : 59 => 3 pages estimées.
Page 2 : 20 commentaires trouvés.
Page 3 : 20 commentaires trouvés.

Total de commentaires récupérés : 60


In [8]:
all_comments

['Ras parfait',
 'Achat de 1500€ de paires, jamais recues, et jamais remboursées. ',
 'Arnaque , paires jamais reçues , plusieurs dunks panda ,',
 "Achat de plus de 1 500 euros de sneakers, aucune paire reçue ! À éviter !!! voir même à fuir d'urgence",
 'Envoie rapide chaussures en parfaite état. Je recommande 😊',
 "je suis très mécontente du suivi de ma commande. En période en Noel, j'ai demandé un échange avec 3 jours de retard et on me l'a refusé . Je ne recommande absolument pas ce site qui ne se soucie absolument pas de satisfaire ses clients .",
 'Rapidité, conformité... je recommande ! nous sommes très satisfaits du produit.Merci beaucoup',
 "J'ai réussi à trouver une paire qui m'as était très compliqué à trouver !Super service client merci Anna pour la réactivité 😉",
 'Site sérieux et fiable. L’article met un peu plus de temps à partir des locaux que sur d’autres sites mais Troicet prend le temps de vérifier l’authenticité du produit et le livre avec son certificat.Je le consei